In [1]:
!pip install datasets
!pip uninstall -y faiss faiss-cpu faiss-gpu
!pip install faiss-cpu
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 11.8 MB/s eta 0:00:00


In [ ]:
import os
import sys

!rm -r /content/goodRAG
os.system('git clone https://github.com/shannn1/goodRAG.git')
sys.path.append('/content/goodRAG')

In [4]:
import torch
from torch.utils.data import DataLoader
import faiss
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
import wandb
from modeling_rag import RagRetriever, RagSequenceForGeneration, RagConfig
from tokenization_rag import RagTokenizer
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DPRQuestionEncoder,
    DPRQuestionEncoderTokenizer,
    AdamW,
    get_scheduler
)
import json
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

In [5]:
knowledge_base = load_dataset("Shannnh/knowledge_base_genai")
knowledge_base = knowledge_base['train']
knowledge_base = knowledge_base.rename_column("document", "text")
output_path = "./knowledge_base"
knowledge_base.save_to_disk(output_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/396 [00:00<?, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/261M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/261M [00:00<?, ?B/s]

...

train-00005-of-00008.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

train-00006-of-00008.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

train-00007-of-00008.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78529 [00:00<?, ? examples/s]

Saving the dataset (0/8 shards):   0%|          | 0/78529 [00:00<?, ? examples/s]

In [6]:
generator = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
generator_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

indexPath = "./faiss_index"
passagesPath = "./knowledge_base"

rag_config = RagConfig.from_question_encoder_generator_configs(
    question_encoder.config,
    generator.config,
    index_name="custom",
    n_docs=5,
    index_path=indexPath,
    passages_path=passagesPath,
)

retriever = RagRetriever(
    config=rag_config,
    question_encoder_tokenizer=question_encoder_tokenizer,
    generator_tokenizer=generator_tokenizer,
)

model = RagSequenceForGeneration(
    config=rag_config,
    question_encoder=question_encoder,
    generator=generator,
    retriever=retriever,
)

retriever.config.index_path = rag_config.index_path
retriever.config.passages_path = rag_config.passages_path
retriever.config.generator = rag_config.generator
retriever.config.question_encoder = rag_config.question_encoder

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
assert model.config.index_path == retriever.config.index_path, "Index path mismatch!"
assert model.config.passages_path == retriever.config.passages_path, "Passages path mismatch!"
assert model.config.generator == retriever.config.generator, "Generator config mismatch!"
assert model.config.question_encoder == retriever.config.question_encoder, "Question encoder config mismatch!"
print("All configurations are consistent!")

All configurations are consistent!


In [8]:
# Initialize wandb
wandb.init(project="rag_finetuning2", name="rag2")

datasets = load_dataset("lighteval/natural_questions_clean")

# Split train and validation
full_train_data = datasets['train'].select(range(20))
train_val_split = full_train_data.train_test_split(test_size=0.1, seed=42)
train_data = train_val_split['train']
validation_data = train_val_split['test']

# Rename columns
train_data = train_data.rename_column("short_answers", "answer")
validation_data = validation_data.rename_column("short_answers", "answer")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

model.to(device)

def preprocess_function(batch):
    question = [q if isinstance(q, str) else "" for q in batch['question']]
    answer = [a[0] if (isinstance(a, list) and len(a) > 0 and isinstance(a[0], str)) else "" for a in batch['answer']]
    inputs = question_encoder_tokenizer(question, padding="max_length", max_length = 512, truncation=True, return_tensors="pt")
    targets = generator_tokenizer(answer, padding="max_length", max_length = 512, truncation=True, return_tensors="pt")

    return {
        "input_ids": inputs["input_ids"].tolist(),
        "attention_mask": inputs["attention_mask"].tolist(),
        "labels": targets["input_ids"].tolist(),
    }

train_dataset = train_data.map(preprocess_function, batched=True)
validation_dataset = validation_data.map(preprocess_function, batched=True)

def collate_fn(batch):
    return {
        "input_ids": torch.tensor([item["input_ids"] for item in batch]),
        "attention_mask": torch.tensor([item["attention_mask"] for item in batch]),
        "labels": torch.tensor([item["labels"] for item in batch]),
        "question": [item["question"] for item in batch],
        "original_answer": [item["answer"] for item in batch],
    }

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
validation_dataloader = DataLoader(validation_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

epochs = 3
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

scaler = GradScaler()

model.train()

for epoch in range(epochs):
    # Training loop
    train_loss_total = 0.0
    train_steps = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            if loss.dim() > 0:
                loss = loss.mean()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        lr_scheduler.step()

        train_loss_total += loss.item()
        train_steps += 1

    # Compute average training loss for the epoch
    avg_train_loss = train_loss_total / train_steps if train_steps > 0 else 0.0

    # Validation loop
    model.eval()
    val_loss = 0
    val_steps = 0
    with torch.no_grad():
        for batch in validation_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.mean().item()
                val_steps += 1

    avg_val_loss = val_loss / val_steps if val_steps > 0 else 0.0

    print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

    # Log epoch losses to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss_epoch": avg_train_loss,
        "val_loss_epoch": avg_val_loss
    })

    model.train()

model.save_pretrained("finetuned_rag")

wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-8-d29c140dc7d1>:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-8-d29c140dc7d1>:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
<ipython-input-8-d29c140dc7d1>:96: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1, Train Loss: 67.92349269655016, Validation Loss: 85.20893478393555


KeyboardInterrupt: 

In [8]:
# Move model to the appropriate device and set to evaluation mode
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# Prepare the test dataset
test_data = datasets['validation'].select(range(5))
test_data = test_data.rename_column("short_answers", "answer")
test_dataset = test_data.map(preprocess_function, batched=True)

test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)
results = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        # Move input data to the appropriate device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Generate predictions
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=50,
            num_beams=5
        )

        # Decode predictions
        predicted_answers = generator_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        # Store results
        for question, original_answer, predicted_answer in zip(batch["question"], batch["original_answer"], predicted_answers):
            results.append({
                "question": question,
                "original_answer": original_answer,
                "predicted_answer": predicted_answer
            })

# Save results to a file
results_file = "results.json"
with open(results_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"Test results saved to {results_file}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Testing:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
Testing: 100%|██████████| 5/5 [01:17<00:00, 15.42s/it]

Test results saved to results.json


In [10]:
def calculate_f1(predicted, ground_truths):
    if not isinstance(ground_truths, list):
        ground_truths = [ground_truths]
    max_f1 = 0.0
    for ground_truth in ground_truths:
        pred_tokens = predicted.split()
        gt_tokens = ground_truth.split()
        common = set(pred_tokens) & set(gt_tokens)
        if len(common) == 0:
            continue
        precision = len(common) / len(pred_tokens)
        recall = len(common) / len(gt_tokens)
        f1 = 2 * precision * recall / (precision + recall)
        max_f1 = max(max_f1, f1)
    return max_f1

f1_scores = [
    calculate_f1(item["predicted_answer"], item["original_answer"])
    for item in results
]
average_f1 = sum(f1_scores) / len(f1_scores)
print(f"Average F1 Score: {average_f1:.4f}")

Average F1 Score: 0.0422


In [11]:
def calculate_exact_match(predicted, ground_truths):
    if not isinstance(ground_truths, list):
        ground_truths = [ground_truths]
    for ground_truth in ground_truths:
        if predicted.strip() == ground_truth.strip():
            return 1
    return 0

em_scores = [
    calculate_exact_match(item["predicted_answer"], item["original_answer"])
    for item in results
]
average_em = sum(em_scores) / len(em_scores)
print(f"Exact Match Score: {average_em:.4f}")

Exact Match Score: 0.0000


In [12]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def calculate_q_bleu(predicted, ground_truths, question):
    if isinstance(ground_truths, list):
        ground_truths = " ".join(ground_truths)

    # Tokenize inputs
    ref_tokens = ground_truths.split()
    hyp_tokens = predicted.split()
    question_tokens = question.split()

    # Calculate BLEU score
    reference = [ref_tokens]
    hypothesis = hyp_tokens
    bleu_score = sentence_bleu(reference, hypothesis)

    # Remove stopwords
    important_ref_tokens = [token for token in ref_tokens if token.lower() not in stop_words]
    important_hyp_tokens = [token for token in hyp_tokens if token.lower() not in stop_words]

    # Key match ratio
    key_match = len(set(important_ref_tokens) & set(important_hyp_tokens)) / max(len(set(important_ref_tokens)), 1)

    # Question match ratio
    question_match = len(set(question_tokens) & set(hyp_tokens)) / max(len(set(question_tokens)), 1)

    # Weighted Q-BLEU score
    q_bleu = 0.7 * bleu_score + 0.2 * key_match + 0.1 * question_match

    return q_bleu

q_bleu_scores = [
    calculate_q_bleu(item["predicted_answer"], item["original_answer"], item["question"])
    for item in results
]
average_q_bleu = sum(q_bleu_scores) / len(q_bleu_scores)
print(f"Average Q-BLEU Score: {average_q_bleu:.4f}")

Average Q-BLEU Score: 0.0125


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over